# 2D Histograms

We start this example by loading in a large galaxy cluster and selecting all the gas cells inside R200c.

In [ ]:
import paicos as pa
import numpy as np
simfolder12 = '/llust21/cosmo-plasm/zoom-simulations/halo_0003/adiabatic-mhd/zoom12/'
simfolder24 = '/llust21/cosmo-plasm/zoom-simulations/halo_0003/adiabatic-mhd/zoom24/'

# A snapshot object
snap = pa.Snapshot(simfolder12 + 'output', 130, load_catalog=False)

# We use the zoom12 catalog
cat = pa.Catalog(simfolder12 + 'output', 130)

# The center of the most massive Friends-of-friends group in the simulation
center = cat.Group['GroupPos'][0]
R200c = cat.Group['Group_R_Crit200'][0]

In [ ]:
# Use OpenMP parallel Cython code
index = pa.util.get_index_of_radial_range(snap['0_Coordinates'],
                                          center, 0., R200c)

# Create a new snap object which only contains the index above
snap = snap.select(index)

We then create the 2D histogram.

In [ ]:
# Create histogram
rhoB = pa.Histogram2D(snap, '0_Density', '0_MagneticFieldStrength', weights='0_Masses', bins_x=200,
                      bins_y=200, logscale=True)

# Create colorlabel
colorlabel = rhoB.get_colorlabel(r'\rho', 'B', 'M')

We can check the Histogram2D doc string for details on the options

In [ ]:
pa.Histogram2D?

We are now ready to make a plot.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


# Make a quick plot
plt.figure(1)
plt.clf()

# Convert to physical and some sensible units
xvals = rhoB.centers_x.to_physical.to('g/cm3')  # xvals = rhoB.centers_x would give comoving code units.
yvals = rhoB.centers_y.to_physical.to('uG') # yvals = rhoB.centers_y would give comoving code units.

plt.pcolormesh(xvals.value, yvals.value,
               rhoB.hist2d.value, norm=LogNorm())
plt.xlabel(xvals.label('\\rho'))
plt.ylabel(yvals.label('B'))

plt.title('paicos Histogram2D')
if rhoB.logscale:
    plt.xscale('log')
    plt.yscale('log')
cbar = plt.colorbar()
cbar.set_label(rhoB.colorlabel)
plt.title('hist2d paicos, pcolormesh')
plt.show()

Let's save the data for later plotting

In [ ]:
rhoB.save(basedir='.', basename='rhoB_hist')

Let's do a density Temperature plot as well

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Create histogram
hist = pa.Histogram2D(snap, '0_Density', '0_Temperatures', weights='0_Masses', bins_x=200,
                      bins_y=200, logscale=True)

# Create colorlabel
colorlabel = hist.get_colorlabel(r'\rho', 'T', 'M')

# Convert to physical and some sensible units
xvals = hist.centers_x.to_physical.to('g/cm3')  # xvals = hist.centers_x would give comoving code units.
yvals = hist.centers_y

# Make a quick plot
plt.figure(1)
plt.clf()

plt.pcolormesh(xvals.value, yvals.value,
               hist.hist2d.value, norm=LogNorm())
plt.xlabel(xvals.label('\\rho'))
plt.ylabel(yvals.label('T'))

plt.title('paicos Histogram2D')
if hist.logscale:
    plt.xscale('log')
    plt.yscale('log')
cbar = plt.colorbar()
cbar.set_label(hist.colorlabel)
plt.title('hist2d paicos, pcolormesh')
plt.show()